The aim of this notebook is to calculate and store most similar superpatches
so that they can be used further for pretraining.

Follow-up to `notebooks/demo_vit.ipynb`.

# Setup, constants & imports

In [ ]:
# TODO: review code

In [ ]:
# MODEL = 'vit_base_patch16_224.dino'
MODEL = 'vit_tiny_patch16_224.augreg_in21k'  # TODO: switch back to vit_base
PATCH_SIZE = 16

N_AGG = 8  # N_AGG x N_AGG patches consitute superpatch
N_NEAREST = 8  # How many nearest neighbors to save

IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 8
NUM_WORKERS = 2
DATA_DIR = '../data/val2017'
VECTORS_PATH = 'vectors_all.npy'  # To store features for all superpatches

VIS_N_SUPERPATCHES_TO_SAMPLE = 8  # How many superpatches to sample for visualization
VIS_N_SIMILAR = 6  # How many most similar superpatches to show per sampled superpatch

# For how many vectors to get sorted nearest neighbors at once in the loop
# It depends on the GPU memory and dataset size
PROCESS_AT_ONCE = 8

In [ ]:
# Run only if using Colab

from google.colab import drive
drive.mount('/content/drive')

! wget http://images.cocodataset.org/zips/train2017.zip
! mkdir data
! unzip -qq train2017.zip -d data

! pip install timm

N_NEAREST = 32
DATA_DIR = 'data/train2017'
VECTORS_PATH = 'drive/My Drive/vectors_all.npy'
PROCESS_AT_ONCE = 256

In [ ]:
import random
import json

from pathlib import Path

import numpy as np
import PIL
import cv2
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import timm

from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

from torch import nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Asserts

In [ ]:
assert IMG_HEIGHT % PATCH_SIZE == 0
assert IMG_WIDTH % PATCH_SIZE == 0
assert IMG_HEIGHT % (N_AGG*PATCH_SIZE) == 0
assert IMG_WIDTH % (N_AGG*PATCH_SIZE) == 0

# Load data

In [ ]:
class ImgsDataset(Dataset):
  def __init__(self, paths, transform):
    self.paths = paths
    self.transform = transform

  def __getitem__(self, index):
    img = PIL.Image.open(self.paths[index]).convert('RGB')
    return self.transform(img)

  def __len__(self):
    return len(self.paths)

In [ ]:
paths = sorted(Path(DATA_DIR).rglob('*'))
paths

In [ ]:
transform = [
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
]

ds = ImgsDataset(paths, transforms.Compose(transform))
loader = DataLoader(
    ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available()
)

In [ ]:
transform_orig = [
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH))
]

ds_orig = ImgsDataset(paths, transforms.Compose(transform_orig))

# Load model

In [ ]:
timm.list_models('vit_tiny_*', pretrained=True)

In [ ]:
model = timm.create_model(
    MODEL,
    pretrained=True,
    img_size=(IMG_HEIGHT, IMG_WIDTH),
    num_classes=0,
    global_pool=''
).to(device).eval()

# Compute vectors

In [ ]:
vectors_all = []

for imgs in tqdm(loader):
    with torch.no_grad():
        # Skip [CLS] token
        vectors = model(imgs.to(device))[:, 1:, :]

        # (B, H*W, C) -> (B, H, W, C)
        vectors = vectors.view(
            vectors.shape[0], IMG_HEIGHT//PATCH_SIZE, IMG_WIDTH//PATCH_SIZE, vectors.shape[2]
        )

        # (B, H, W, C) -> (B, C, H, W)
        vectors = vectors.permute(0, 3, 1, 2)

        # Get superpatches representations
        vectors = nn.AvgPool2d(N_AGG, count_include_pad=False)(vectors)

        # (B, C, H, W) -> (B, H, W, C)
        vectors = vectors.permute(0, 2, 3, 1)

        # (B, H, W, C) -> (B*H*W, C)
        # Get flattened superpatches representations
        vectors = vectors.flatten(0, 2)

        vectors_all.append(vectors.cpu().squeeze().numpy())

del model
vectors_all = np.vstack(vectors_all)
vectors_all.shape

In [ ]:
np.save(VECTORS_PATH, vectors_all) 

# Compute nearest neighbors

In [ ]:
vectors_all = np.load(VECTORS_PATH)
vectors_all = torch.from_numpy(vectors_all).to(device)  # Move back to GPU if available

In [ ]:
@torch.no_grad()
def get_similarities(a, b, eps=1e-8):
    norm_a, norm_b = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]

    a_norm = a / torch.max(norm_a, eps*torch.ones_like(norm_a))
    b_norm = b / torch.max(norm_b, eps*torch.ones_like(norm_b))

    similarities = torch.mm(a_norm, b_norm.transpose(0, 1))

    return similarities

In [ ]:
nns_dict = {}

n_iters = vectors_all.shape[0] // PROCESS_AT_ONCE + 1
for iter_idx in tqdm(range(n_iters)):
    start_idx = int(iter_idx*PROCESS_AT_ONCE)
    end_idx = int(min(start_idx+PROCESS_AT_ONCE, vectors_all.shape[0]))

    similarities = get_similarities(
        vectors_all[start_idx : end_idx], vectors_all
    )

    most_similar = torch.argsort(
        similarities, dim=1, descending=True
    )[:, :N_NEAREST].cpu().tolist()  # `1:N_NEAREST+1` to skip identical crop

    for i in range(len(most_similar)):
        nns_dict[start_idx+i] = most_similar[i]

with open('nns.json', 'w') as f:
    json.dump(nns_dict, f)

# Visualize results

In [ ]:
with open('nns.json', 'r') as f:
    nns_dict = json.load(
        f,
        # Convert keys to integers during loading
        object_pairs_hook=lambda x: {int(k): v for k, v in x}
    )

In [ ]:
def overlay_superpatch_by_index(superpatch_idx):
    n_superpatch_cols_per_img = IMG_WIDTH // PATCH_SIZE // N_AGG
    n_superpatch_rows_per_img = IMG_HEIGHT // PATCH_SIZE // N_AGG
    n_superpatchs_per_img = n_superpatch_cols_per_img * n_superpatch_rows_per_img

    img_idx = superpatch_idx // n_superpatchs_per_img
    local_superpatch_idx = superpatch_idx % n_superpatchs_per_img
    superpatch_row_idx = local_superpatch_idx // n_superpatch_cols_per_img
    superpatch_col_idx = local_superpatch_idx % n_superpatch_cols_per_img

    y_upper = superpatch_row_idx * PATCH_SIZE * N_AGG
    x_upper = superpatch_col_idx * PATCH_SIZE * N_AGG
    y_lower = (superpatch_row_idx+1) * PATCH_SIZE * N_AGG
    x_lower = (superpatch_col_idx+1) * PATCH_SIZE * N_AGG

    img_w_crop = np.copy(ds_orig[img_idx])
    cv2.rectangle(
        img_w_crop,
        (x_upper, y_upper),
        (x_lower, y_lower),
        (0, 255, 0)
    )

    return img_w_crop

In [ ]:
superpatch_idxs = random.sample(
    list(nns_dict.keys()),
    VIS_N_SUPERPATCHES_TO_SAMPLE
)

for superpatch_idx in superpatch_idxs:
    fig, axs = plt.subplots(
        1, VIS_N_SIMILAR+1, figsize=(VIS_N_SIMILAR*3, 5)
    )
    for ax in axs:
        ax.axis('off')

    axs[0].imshow(overlay_superpatch_by_index(superpatch_idx))
    axs[0].set_title('Sampled superpatch')
    for i in range(VIS_N_SIMILAR):
        axs[i+1].imshow(overlay_superpatch_by_index(nns_dict[superpatch_idx][i]))
        axs[i+1].set_title(f'Most similar {i+1}')
    plt.show()
    plt.close()